# Use PCA For Dimensionality Reduction

In this example we will use Principal Components Analysis (PCA) to reduce the dimensionality in the feature set from 4 to 2 in a way that best explains the output variance.

In [1]:
import (
    "fmt"
    "github.com/kniren/gota/dataframe"
    "github.com/kniren/gota/series"
    "io/ioutil"
    "bytes"
)

In [2]:
const path = "../datasets/iris/iris.csv"

In [3]:
b, err := ioutil.ReadFile(path)
if err != nil {
    fmt.Println("Error!", err)
}
df := dataframe.ReadCSV(bytes.NewReader(b))
df.SetNames("petal length", "petal width", "sepal length", "sepal width", "species")

In [4]:
df

[150x5] DataFrame

    petal length petal width sepal length sepal width species
 0: 5.100000     3.500000    1.400000     0.200000    0      
 1: 4.900000     3.000000    1.400000     0.200000    0      
 2: 4.700000     3.200000    1.300000     0.200000    0      
 3: 4.600000     3.100000    1.500000     0.200000    0      
 4: 5.000000     3.600000    1.400000     0.200000    0      
 5: 5.400000     3.900000    1.700000     0.400000    0      
 6: 4.600000     3.400000    1.400000     0.300000    0      
 7: 5.000000     3.400000    1.500000     0.200000    0      
 8: 4.400000     2.900000    1.400000     0.200000    0      
 9: 4.900000     3.100000    1.500000     0.100000    0      
    ...          ...         ...          ...         ...    
    <float>      <float>     <float>      <float>     <int>  


In [5]:
//  Standardise maps the given column values by subtracting mean and rescaling by standard deviation
func Standardise(df dataframe.DataFrame, col string) dataframe.DataFrame {
    s := df.Col(col)
    std := s.StdDev()
    mean := s.Mean()
    v := make([]float64, s.Len(), s.Len())
    for i := 0; i < s.Len(); i++ {
        v[i] = (s.Elem(i).Float()-mean)/std
    }
    rs := series.Floats(v)
    rs.Name = col
    return df.Mutate(rs)
}

In [6]:
//  DropColumn returns a new dataframe that does not include the given column
func DropColumn(df dataframe.DataFrame, col string) dataframe.DataFrame {
    var s []series.Series
    for _, c := range df.Names() {
        if c == col {
            continue
        }
        s = append(s, df.Col(c))
    }
    return dataframe.New(s...)
}

In [7]:
df = Standardise(df, "petal length")
df = Standardise(df, "petal width")
df = Standardise(df, "sepal length")
df = Standardise(df, "sepal width")
labels := df.Col("species").Float()
df = DropColumn(df, "species")

In [8]:
df

[150x4] DataFrame

    petal length petal width sepal length sepal width
 0: -0.897674    1.015602    -1.335752    -1.311052  
 1: -1.139200    -0.131539   -1.335752    -1.311052  
 2: -1.380727    0.327318    -1.392399    -1.311052  
 3: -1.501490    0.097889    -1.279104    -1.311052  
 4: -1.018437    1.245030    -1.335752    -1.311052  
 5: -0.535384    1.933315    -1.165809    -1.048667  
 6: -1.501490    0.786174    -1.335752    -1.179859  
 7: -1.018437    0.786174    -1.279104    -1.311052  
 8: -1.743017    -0.360967   -1.335752    -1.311052  
 9: -1.139200    0.097889    -1.279104    -1.442245  
    ...          ...         ...          ...        
    <float>      <float>     <float>      <float>    


In [9]:
import (
    "github.com/gonum/matrix/mat64"
)

In [10]:
//  DataFrameToMatrix converts the given dataframe to a gonum matrix
func DataFrameToMatrix(df dataframe.DataFrame) mat64.Matrix {
    var x []float64 //slice to hold matrix entries in row-major order
    
    for i := 0; i < df.Nrow(); i++ {
        for j := 0; j < df.Ncol(); j ++ {
            x = append(x, df.Elem(i,j).Float())
        }    
    }
    return mat64.NewDense(df.Nrow(), df.Ncol(), x)
}

In [11]:
features := DataFrameToMatrix(df)

# Create PCA

Now that we have pre-processed the data we will perform PCA on the features matrix, keeping the first 2 components.

In [12]:
import (
    "gonum.org/v1/plot"
    "gonum.org/v1/plot/plotter"
    "gonum.org/v1/plot/plotutil"
    "gonum.org/v1/plot/vg"
    "github.com/cdipaolo/goml/cluster"
    "github.com/cdipaolo/goml/base"
    "github.com/gonum/stat"
    "bufio"
    "strconv"
)

In [13]:
model := stat.PC{}
if ok := model.PrincipalComponents(features, nil); !ok {
    fmt.Println("Error!")
}

In [14]:
variances := model.Vars(nil)
components := model.Vectors(nil)

In [15]:
// Print the amount of variance explained by each component
total_variance := 0.0
for i := range variances {
    total_variance += variances[i]
}

for i := range variances {
    fmt.Printf("Component %d: %5.3f\n", i + 1, variances[i] / total_variance)
}

Component 1: 0.730
Component 2: 0.229
Component 3: 0.037
Component 4: 0.005


In [16]:
// Transform the data into the new space
transform := mat64.NewDense(df.Nrow(), 4, nil)
transform.Mul(features, components)

In [17]:
//  PCA keeps top 2 components (one for x axis, one for y) and returns this as map from label to XYs
//  Matrix must have at least 2 columns or this will panic
func PCAToScatterData(m mat64.Matrix, labels []float64) map[int]plotter.XYs {
    ret := make(map[int]plotter.XYs)
    nrows, _ := m.Dims()
    for i := 0; i < nrows; i++ {
        var pt struct{X, Y float64}
        pt.X = m.At(i, 0)
        pt.Y = m.At(i, 1)
        ret[int(labels[i])] = append(ret[int(labels[i])], pt)
    }
    return ret
} 

In [18]:
/**
    NB. This is required because gophernotes comes with an old version of goml. When it gets updated we can remove most of this.
*/

type LegacyXYs plotter.XYs

func (xys LegacyXYs) Len() int {
    return len(xys)
}

func (xys LegacyXYs) XY(i int) (float64, float64) {
    return xys[i].X, xys[i].Y
}

In [19]:
scatterData := PCAToScatterData(transform, labels)

In [20]:
func PlotPCAData(labelsToXYs map[int]plotter.XYs, xLabel, yLabel string) ([]uint8, error) {
    p, err := plot.New()
    if err != nil {
        return nil, err
    }
    p.Title.Text = "Iris Dataset PCA Example"
    //p.X.Min = 4
    //p.X.Max = 9
    p.X.Padding = 0
    p.X.Label.Text = xLabel
    //p.Y.Min = 1.5
    //p.Y.Max = 4.5
    p.Y.Padding = 0
    p.Y.Label.Text = yLabel
    for i := range labelsToXYs {
        s, err := plotter.NewScatter(LegacyXYs(labelsToXYs[i])) //Remove LegacyXYs when gophernotes updated to use latest goml
        s.Color = plotutil.Color(i)
        s.Shape = plotutil.Shape(i)
        p.Add(s)
        n := strconv.Itoa(i)
        p.Legend.Add(n)
        if err != nil {
            return nil, err
        }
    }
    w, err := p.WriterTo(5*vg.Inch, 4*vg.Inch, "jpg")
    if err != nil{
        return nil, err
    }
    if err := p.Save(5*vg.Inch, 4*vg.Inch, "PCA Scatter.jpg"); err != nil {
        return nil, err
    }
    var b bytes.Buffer
    writer := bufio.NewWriter(&b)
    w.WriteTo(writer)
    return b.Bytes(), nil
}

In [21]:
b, _ := PlotPCAData(scatterData, "Component 1", "Component 2")
display.JPEG(b)